In [1]:
library(twang)
library(weights)
library(tidyverse)

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:latticeExtra’:

    layer


Attaching package: ‘Hmisc’

The following objects are masked from ‘package:xtable’:

    label, label<-

The following object is masked from ‘package:survey’:

    deff

The following objects are masked from ‘package:base’:

    format.pval, round.POSIXt, trunc.POSIXt, units

Loading required package:

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds")) %>%
    mutate(sunday = factor(as.integer(icu_adm_weekday == 0), levels = c(0, 1)),
           monday = factor(as.integer(icu_adm_weekday == 1), levels = c(0, 1)),
           tuesday = factor(as.integer(icu_adm_weekday == 2), levels = c(0, 1)),
           wednesday = factor(as.integer(icu_adm_weekday == 3), levels = c(0, 1)),
           thursday = factor(as.integer(icu_adm_weekday == 4), levels = c(0, 1)),
           friday = factor(as.integer(icu_adm_weekday == 5), levels = c(0, 1)),
           saturday = factor(as.integer(icu_adm_weekday == 6), levels = c(0, 1))) %>%
    mutate(female = factor(as.integer(as.integer(as.character(gender)) == 0), levels = c(0, 1)))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,mort_28_day_int,ps,sunday,monday,tuesday,wednesday,thursday,friday,saturday,female
201220,125078,66690,0,2106-04-27 09:47:50,2106-05-01 19:25:46,1,62.67646,1,NA,⋯,1,0.4447784,0,0,1,0,0,0,0,0
215842,151232,11663,0,2188-02-14 09:48:15,2188-02-16 03:02:48,1,86.76186,1,NA,⋯,0,0.1580691,0,0,0,0,1,0,0,1
234312,164444,86645,1,2165-06-22 09:47:16,2165-07-07 21:55:20,1,56.08904,1,NA,⋯,0,0.3088600,0,0,0,0,0,0,1,0
289157,146726,10304,0,2156-06-23 22:26:00,2156-06-30 17:26:00,1,45.91093,1,2156-06-25 08:00:00,⋯,0,0.7000591,0,0,0,1,0,0,0,0
211964,160170,94534,0,2160-03-05 22:23:19,2160-03-07 06:48:41,1,59.38693,1,NA,⋯,0,0.3324814,0,0,0,1,0,0,0,0
230173,112553,31544,0,2140-01-31 04:39:25,2140-02-03 02:41:39,1,300.00345,1,NA,⋯,1,0.2379659,0,0,0,0,0,0,1,1


In [4]:
library(tableone)

In [5]:
tab_features <- CreateTableOne(vars = c("vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
                                        "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
                                        "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
                                        "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
                                        "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
                                        "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               strata = "echo",
                               factorVars = c("lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               data = full_data)
tab_features$ContTable

                                   Stratified by echo
                                    0               1                p     
  n                                 3099            3063                   
  vs_map_first (mean (sd))           79.63 (19.44)  119.76 (2183.54)  0.309
  vs_heart_rate_first (mean (sd))    92.97 (19.87)   95.21 (26.70)   <0.001
  vs_temp_first (mean (sd))          36.75 (1.05)    36.83 (1.58)     0.014
  vs_cvp_first (mean (sd))           11.88 (17.09)   13.57 (19.12)    0.034
  lab_wbc_first (mean (sd))          13.48 (14.03)   13.91 (12.58)    0.215
  lab_hemoglobin_first (mean (sd))   10.55 (1.98)    10.65 (2.04)     0.062
  lab_platelet_first (mean (sd))    223.96 (134.79) 213.44 (127.53)   0.002
  lab_sodium_first (mean (sd))      139.09 (6.45)   138.62 (5.92)     0.003
  lab_potassium_first (mean (sd))     4.10 (0.79)     4.15 (0.85)     0.024
  lab_bicarbonate_first (mean (sd))  22.28 (5.40)    22.26 (5.71)     0.864
  lab_chloride_first (mean (sd))  

In [6]:
tab_basics <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                                      "saps", "sofa", "vent", "vaso", "sedative",
                                      "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                      "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                      "icu_adm_weekday"),
                             strata = "echo",
                             factorVars = c("gender", "vent", "vaso", "sedative",
                                            "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                            "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                            "icu_adm_weekday"),
                             data = full_data) %>%
    print

                        Stratified by echo
                         0             1             p      test
  n                       3099          3063                    
  age (mean (sd))        84.18 (67.14) 79.17 (59.72)  0.002     
  gender = 1 (%)          1524 (49.2)   1603 (52.3)   0.014     
  weight (mean (sd))      1.80 (0.74)   1.84 (0.77)   0.061     
  elix_score (mean (sd))  8.51 (7.45)  10.05 (7.68)  <0.001     
  saps (mean (sd))       19.63 (5.79)  20.78 (5.45)  <0.001     
  sofa (mean (sd))        5.30 (3.62)   6.32 (3.80)  <0.001     
  vent = 1 (%)            1459 (47.1)   1808 (59.0)  <0.001     
  vaso = 1 (%)             839 (27.1)   1176 (38.4)  <0.001     
  sedative = 1 (%)        1247 (40.2)   1540 (50.3)  <0.001     
  icd_chf = 1 (%)          563 (18.2)   1198 (39.1)  <0.001     
  icd_afib = 1 (%)         622 (20.1)    988 (32.3)  <0.001     
  icd_renal = 1 (%)        436 (14.1)    503 (16.4)   0.011     
  icd_liver = 1 (%)        314 (10.1)    340 (1

In [7]:
table_one_raw <- c("age", "female", "weight", "elix_score",
  "saps", "sofa", "vent", "vaso", "sedative",
  "icd_chf", "icd_afib", "icd_renal", "icd_liver",
  "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
  "sunday", "monday", "tuesday", "wednesday",
  "thursday", "friday", "saturday",
  "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
  "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
  "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
  "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
  "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
  "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag") %>%
map(function(var) {
    x <- full_data[[var]]
    sym_var <- rlang::sym(var)
    if (is.numeric(x)) {
        res <- full_data %>%
            group_by(echo) %>%
            summarise(main = mean(!!sym_var, na.rm = TRUE),
                      sub = sd(!!sym_var, na.rm = TRUE),
                      wtd_main = wtd.mean(!!sym_var, weight),
                      wtd_sub = sqrt(wtd.var(!!sym_var, weight))) %>%
            cbind(
                full_data %>%
                { split(.[[var]], .$echo) } %>%
                setNames(c("x", "y")) %>%
                do.call(t.test, .) %>%
                .$p.value %>%
                data.frame(p_val = .)
            ) %>%
            cbind(
                full_data %>%
                { split(select(., !!sym_var, weight), .$echo) } %>%
                setNames(c("fst", "sec")) %>%
                {
                    list(x = { pull(pluck(., "fst"), !!sym_var) },
                         y = { pull(pluck(., "sec"), !!sym_var) },
                         weight = { pull(pluck(., "fst"), weight) },
                         weighty = { pull(pluck(., "sec"), weight) },
                         samedata = FALSE) 
                }
                %>%
                do.call(wtd.t.test, .) %>%
                .$coefficients %>%
                .["p.value"] %>%
                unname %>%
                data.frame(wtd_p_val = .)
            )
    }
    
    if (is.factor(x)) {
        if (length(levels(x)) == 2) {
            res <- full_data %>%
                mutate(!!sym_var := as.integer(as.character(!!sym_var))) %>%
                group_by(echo) %>%
                summarise(main = mean(!!sym_var, na.rm = TRUE) * 100,
                          sub = NA,
                          wtd_main = wtd.mean(!!sym_var, weight) * 100,
                          wtd_sub = NA) %>%
                cbind(
                    full_data %>%
                    {
                        list(x = pull(., !!sym_var),
                             y = pull(., echo))
                    } %>%
                    do.call(chisq.test, .) %>%
                    pluck("p.value") %>%
                    unname %>%
                    data.frame(p_val = .)
                ) %>%
                cbind(
                    full_data %>%
                    {
                        list(var1 = pull(., !!sym_var),
                             var2 = pull(., echo),
                             weight = pull(., weight))
                    } %>%
                    do.call(wtd.chi.sq, .) %>%
                    pluck("p.value") %>%
                    unname %>%
                    data.frame(wtd_p_val = .)
                )
        } else return(NULL)
    }
    
    res %>%
    gather("key", "value", main, sub, wtd_main, wtd_sub) %>%
    unite("key", key, echo) %>%
    spread(key, value) %>%
    mutate(var = var) %>%
    return
}) %>%
discard(is.null) %>%
data.table::rbindlist() %>%
as.data.frame

head(table_one_raw)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
1.942281e-03,0.04772662,84.183688,79.166557,67.137454,59.7201585,83.089515,79.918203,64.906984,60.763406,age
1.416420e-02,0.34205147,50.822846,47.665687,NA,NA,49.415944,48.205909,NA,NA,female
6.091011e-02,0.03565708,1.801389,1.837418,0.738580,0.7696856,2.104114,2.159730,1.036885,1.040762,weight
1.705518e-15,0.08790062,8.505970,10.045054,7.451124,7.6816088,9.106113,9.436513,7.555664,7.638541,elix_score
1.378591e-15,0.12425987,19.632462,20.778975,5.788347,5.4462928,20.019478,20.236554,5.657332,5.425510,saps
7.217556e-27,0.02783258,5.304937,6.323865,3.618100,3.7963648,5.662260,5.868751,3.669506,3.697442,sofa


In [8]:
table_one_fmt <- table_one_raw %>%
mutate(raw_0 = ifelse(is.na(sub_0), sprintf("%.2f%%", main_0),
                      sprintf("%.2f (%.2f)", main_0, sub_0)),
       raw_1 = ifelse(is.na(sub_1), sprintf("%.2f%%", main_1),
                      sprintf("%.2f (%.2f)", main_1, sub_1)),
       wtd_raw_0 = ifelse(is.na(wtd_sub_0), sprintf("%.2f%%", wtd_main_0),
                          sprintf("%.2f (%.2f)", wtd_main_0, wtd_sub_0)),
       wtd_raw_1 = ifelse(is.na(wtd_sub_1), sprintf("%.2f%%", wtd_main_1),
                          sprintf("%.2f (%.2f)", wtd_main_1, wtd_sub_1)),
       p_val = ifelse(p_val < 0.001, "<0.001", as.character(round(p_val, 3))),
       wtd_p_val = ifelse(wtd_p_val < 0.001, "<0.001", as.character(round(wtd_p_val, 3)))) %>%
select(var, raw_1, raw_0, p_val, wtd_raw_1, wtd_raw_0, wtd_p_val) %>%
mutate(var = gsub("vs_", "", var)) %>%
mutate(var = gsub("lab_", "", var)) %>%
mutate(var = gsub("icd_", "", var)) %>%
mutate(var = gsub("_flag", "", var)) %>%
mutate(var = gsub("_first", "", var)) %>%
mutate(var = gsub("_", " ", var)) %>%
mutate(var = tools::toTitleCase(var)) %>%
setNames(c("Covariate", "Echo", "Non-Echo", "p value",
           "Echo (weighted cohort)", "Non-Echo (weighted cohort)",
           "p value (weighted cohort)"))

table_one_fmt

Covariate,Echo,Non-Echo,p value,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort)
Age,79.17 (59.72),84.18 (67.14),0.002,79.92 (60.76),83.09 (64.91),0.048
Female,47.67%,50.82%,0.014,48.21%,49.42%,0.342
Weight,1.84 (0.77),1.80 (0.74),0.061,2.16 (1.04),2.10 (1.04),0.036
Elix Score,10.05 (7.68),8.51 (7.45),<0.001,9.44 (7.64),9.11 (7.56),0.088
Saps,20.78 (5.45),19.63 (5.79),<0.001,20.24 (5.43),20.02 (5.66),0.124
Sofa,6.32 (3.80),5.30 (3.62),<0.001,5.87 (3.70),5.66 (3.67),0.028
Vent,59.03%,47.08%,<0.001,54.17%,50.69%,0.006
Vaso,38.39%,27.07%,<0.001,33.97%,30.81%,0.008
Sedative,50.28%,40.24%,<0.001,46.27%,43.05%,0.011
Chf,39.11%,18.17%,<0.001,30.34%,25.42%,<0.001


In [9]:
data.table::fwrite(table_one_fmt, file.path(data_dir, "tableone.csv"))
data.table::fwrite(table_one_raw, file.path(data_dir, "tableone_raw.csv"))